In [249]:
import pandas as pd
import requests
import json
import datetime

In [250]:
gare = "Montpellier Saint-Roch"

# Max 250
nb_trains = 20

In [251]:
token = 'e7b7fedd-71d0-48c6-8cc7-749e22ba8e80'

## Dictionnaire

In [252]:
req_gare = requests.get("https://ressources.data.sncf.com/api/records/1.0/search/?dataset=referentiel-gares-voyageurs&q=&rows=3220&sort=gare_alias_libelle_noncontraint&facet=departement_libellemin&facet=segmentdrg_libelle&facet=gare_agencegc_libelle&facet=gare_regionsncf_libelle&facet=gare_ug_libelle")
doc_gare = json.loads(req_gare.text)
row_gare = len(doc_gare['records'])
print(f'Nombre de lignes : {row_gare}')

Nombre de lignes : 3220


In [253]:
df_dic = pd.DataFrame(doc_gare['records'])
df_dic = pd.DataFrame(list(df_dic['fields']))
df_dic = df_dic[['alias_libelle_noncontraint','uic_code']]

In [254]:
def removezero(string):
    return 'SNCF:' + str(string)[2:]

df_dic['uic_code'] = df_dic['uic_code'].apply(removezero)

In [255]:
df_dic.convert_dtypes()
dic_gare = df_dic.set_index('alias_libelle_noncontraint').T.to_dict('list')

/var/folders/bg/vbvjjnmj0hv9wmb92ngxcq9m0000gn/T/ipykernel_6046/3118890846.py:2: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  dic_gare = df_dic.set_index('alias_libelle_noncontraint').T.to_dict('list')


## Début

In [256]:
link = 'https://api.sncf.com/v1/coverage/sncf/stop_areas/stop_area:' + dic_gare[gare][0] + '/departures?count=' + str(nb_trains)
req = requests.get(link,auth=(token, ''))
#print(link)

In [257]:
doc = json.loads(req.text)
row = len(doc['departures'])
print(f'Nombre de lignes : {row}')

Nombre de lignes : 20


In [258]:
df = pd.DataFrame(doc['departures'])
df_gare = pd.DataFrame(list(df['display_informations']))
df_heure = pd.DataFrame(list(df['stop_date_time']))
df_id = pd.DataFrame(list(df['links']))
df_id = pd.DataFrame(list(df_id[1]))

In [259]:
def del_day(string):
    return string[9:]

df_heure['departure_date_time'] = df_heure['departure_date_time'].apply(del_day)
df_heure['base_departure_date_time'] = df_heure['base_departure_date_time'].apply(del_day)

In [260]:
def conv_min(string):
    return int(string[0:2])*60 + int(string[2:4])

df_heure['retard'] = df_heure['departure_date_time'].apply(conv_min) - df_heure['base_departure_date_time'].apply(conv_min)

In [261]:
def str_tps(str):
    return datetime.datetime.strptime(str, '%H%M%S').time()

df_heure['heure'] = df_heure['departure_date_time'].apply(str_tps)
df_heure['old_heure'] = df_heure['base_departure_date_time'].apply(str_tps)

In [262]:
def del_par(string):
    index = string.find("(")
    return string[:index]

df_gare['direction'] = df_gare['direction'].apply(del_par)

In [263]:
df_gare = df_gare[['direction','network','trip_short_name']]
df_gare.rename(columns = {'direction':'Destination'}, inplace = True)
df_gare.rename(columns = {'network':'Train'}, inplace = True)
df_gare.rename(columns = {'trip_short_name':'Numéro'}, inplace = True)

df_gare['Départ (réel)'] = df_heure['heure']
df_gare['Départ (prévu)'] = df_heure['old_heure']
df_gare['Retard (min)'] = df_heure['retard']
df_gare['id'] = df_id['id']

In [264]:
def get_name(string):
    string = string[10:]
    index_fin = string.find("', 'links'")
    return string[:index_fin]

arrets = []
causes = []

for i in df_gare.index:
    id = df_gare.iloc[i]['id']
    
    if 'RealTime' in id:
        index_id = id.index("RealTime")
        id = id[:index_id-1]

    link_voyage = 'https://api.sncf.com/v1/coverage/sncf/vehicle_journeys/' + id
    req_arret = requests.get(link_voyage ,auth=(token, ''))
    doc_voyage = json.loads(req_arret.text)

    df_arret = pd.DataFrame(doc_voyage['vehicle_journeys'])
    df_arret = pd.DataFrame(list(df_arret['stop_times']))
    df_arret = df_arret.T
    df_arret = pd.DataFrame(list(df_arret[0]))
    df_arret['stop_point'] = df_arret['stop_point'].astype('str').apply(get_name)

    if df_gare.iloc[i]['Retard (min)'] != 0:
        df_retard = pd.DataFrame(doc_voyage['disruptions'])
        if 'messages' in df_retard:
            df_retard = pd.DataFrame(list(df_retard['messages'][0]))
            causes.append(df_retard.iloc[0]['text'])
        else:
            causes.append("Retard non expliqué")
    else:
        causes.append("")

    liste_arrets = list(df_arret['stop_point'])
    if gare not in liste_arrets:
        if "Paris Gare de Lyon" in liste_arrets:
            index_gare = liste_arrets.index("Paris Gare de Lyon")
        if "Paris - Gare de Lyon - Banlieue" in liste_arrets:
            index_gare = liste_arrets.index("Paris - Gare de Lyon - Banlieue")
        if "Gare du Nord Surface" in liste_arrets:
            index_gare = liste_arrets.index("Gare du Nord Surface")
    else:
        index_gare = liste_arrets.index(gare)

    liste_arrets = liste_arrets[index_gare+1:]

    arrets.append(liste_arrets)

In [265]:
df_gare['Cause'] = causes
df_gare['Arrêts'] = arrets
df_gare = df_gare.drop(['id'], axis=1)

In [266]:
df_gare

,Destination,Train,Numéro,Départ (réel),Départ (prévu),Retard (min),Cause,Arrêts
0,Lyon Part Dieu,TGV INOUI,6873,17:59:30,17:59:30,0,,"[Nîmes Centre, Valence TGV Rhône-Alpes Sud, Ly..."
1,Nîmes Centre,TER,876318,18:05:00,18:05:00,0,,"[Saint-Aunès, Baillargues, Valergues - Lansarg..."
2,Avignon Centre,TER,876424,18:14:00,18:14:00,0,,"[Baillargues, Lunel, Vergèze - Codognan, Nîmes..."
3,Narbonne,TER,876562,18:37:00,18:37:00,0,,"[Villeneuve-lès-Maguelone, Vic - Mireval, Fron..."
4,Nîmes Centre,TER,876620,18:38:00,18:38:00,0,,"[Baillargues, Lunel, Vergèze - Codognan, Nîmes..."
5,Lyon Part Dieu,OUIGO,7810,18:54:00,18:54:00,0,,"[Nîmes Centre, Valence TGV Rhône-Alpes Sud, Ly..."
6,Cerbère,TER,876427,18:55:00,18:55:00,0,,"[Frontignan, Sète, Agde, Béziers, Narbonne, Po..."
7,Marseille - Saint-Charles,Intercités,4663,19:03:00,18:58:00,5,Travaux sur les voies,"[Nîmes Centre, Marseille - Saint-Charles]"
8,Bordeaux Saint-Jean,Intercités,4766,19:07:00,19:07:00,0,,"[Béziers, Narbonne, Carcassonne, Toulouse Mata..."
9,Avignon Centre,TER,86994,19:13:00,19:13:00,0,,"[Baillargues, Lunel, Vergèze - Codognan, Nîmes..."
